# Table of contents

1. [Introduction](#1)
2. [Starting Point](#2) <br>
    2.1 [Constants](#2.1) <br>
    2.2 [Fundamental Ship Parameters](#2.2) <br>
    2.3 [Voyage Ship Parameters](#2.3) <br>
    2.4 [Model Formulas](#2.4) <br>
3. [Optimization](#3) <br>
    3.1 [Initial Guess | x0](#3.1) <br>
    3.2 [Constraints | constraints](#3.2) <br>
    3.3 [Boundaries | bounds](#3.3) <br>
    3.4 [Objective Function | func](#3.4) <br>
    3.5 [Results](#3.5) <br>

## 1. Introduction <a name="1"></a>

The purpose of this exercise is to optimize the main dimensions and speed of service of a bulk carrier, in order to minimize the Annual Unit Load Shipping cost ($/ton). To solve the exercise, the computational model used - a set of empirical and semi-empirical formulas - is the one proposed by Xuebin (2009), as presented in the image below:

<br />
<img src="./images/computational_model_xuebin.png" width="500" /> 
<br />

With the following given constraints:

<br />
<img src="./images/constrains.png" width="200" /> 
<br />


To apply the optimization, we set as our goal to minimize the transportation cost, which is the ratio of annual costs to annual cargo (i.e, transportation cost = annual costs/annual cargo). To solve this problem with programming, we use the Python Scipy package, which provides fundamental algorithms for scientific computing in Python. Specifically, we use the minimize function provided by this package, whose documentation can be found at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

If any error indicating the need to install the Scipy packages occurs when running this notebook, uncomment the final part of the cell below and run it to install it.

In [1]:
# The sys module provides various functions and variables that are used to
# manipulate different parts of the Python runtime environment.
# This provides a safer way to install packages.

# import sys
# !{sys.executable} -m pip install scipy

## 2. Starting Point <a name="2"></a>

Before actually using the minimize function to apply the optimization, we define the set of relevant constants and parameters with their respective initial values:

### 2.1 Constants <a name="2.1"></a>

In [14]:
g = 9.805  # Gravitational acceleration at the earth's surface
p = 1.025  # Sea water density

### 2.2 Fundamental Ship Parameters <a name="2.2"></a>

In [15]:
L = 55 # Length
B = 5 # Breadth
D = 10 # Depth
T = 5 # Draught
Cb = 0.7 # Block Coefficient
Vk = 14 # TODO - what is Vk?
DWT = 100000 # Deadweight

### 2.3 Voyage Parameters  <a name="2.3"></a>

In [16]:
round_trip_miles = 5000
handling_rate = 8000
fuel_price = 100

### 2.4 Model Formulas  <a name="2.4"></a>

To avoid code repetition, we defined functions that calculate parameters to formulas defined by the computational model used:

In [17]:
def get_V(Vk):
    return 0.5144*Vk


def get_displacement(L, B, T, Cb):
    return p*L*B*T*Cb


def get_froude_number(Vk, L):
    V = get_V(Vk)
    return V / ((g*L) ** 0.5)


def get_power(Cb, displacement, V, froude_number):
    a = 4977.06*(Cb ** 2) - 8105.61*Cb + 4456.51
    b = -10847.2*(Cb ** 2) + 12817*Cb - 6960.32
    return (displacement ** (2/3))*(V ** 3) / (a + b*froude_number)


def get_steel_weight(L, B, D, Cb):
    return 0.034*(L**1.7)*(B**0.7)*(D**0.4)*(Cb**0.5)


def get_outfit_weight(L, B, D, Cb):
    return (L ** 0.8)*(B ** 0.6)*(D ** 0.3)*(Cb ** 0.1)


def get_machinery_weight(power): return 0.17*(power ** 0.9)


def get_lightship_weight(steel_weight, outfit_weight, machinery_weight):
    return steel_weight + outfit_weight + machinery_weight


def get_sea_days(round_trip_miles, Vk):
    return round_trip_miles/(24*Vk)


def get_daily_consumption(power):
    return 0.19*power*24/1000 + 0.2


def get_fuel_carried(daily_consumption, sea_days):
    return daily_consumption*(sea_days+5)


def get_miscellaneous_DWT(DWT):
    return 2*(DWT ** 0.5)


def get_cargo_DWT(DWT, fuel_carried, miscellaneous_DWT):
    return DWT-fuel_carried-miscellaneous_DWT


def get_port_days(cargo_DWT, handling_rate):
    return 2*((cargo_DWT/handling_rate)+0.5)


def get_round_trips_per_year(sea_days, port_days):
    return 350/(sea_days+port_days)


def get_port_cost(DWT):
    return 6.3*(DWT ** 0.8)


def get_fuel_cost(daily_consumption, sea_days, fuel_price):
    return 1.05*daily_consumption*sea_days*fuel_price


def get_voyage_costs(fuel_cost, port_cost, round_trips_per_year):
    return (fuel_cost+port_cost)*round_trips_per_year


def get_running_cost(DWT):
    return 40000*(DWT ** 0.3)


def get_ship_cost(steel_weight, outfit_weight, power):
    return 1.3*(2000*steel_weight + 3500 *
                outfit_weight + 2400*(power ** 0.8))


def get_capital_costs(ship_cost): return 0.2*ship_cost


def get_annual_cost(capital_costs, running_cost, voyage_costs):
    return capital_costs + running_cost + voyage_costs


def get_annual_cargo(cargo_DWT, round_trips_per_year):
    return cargo_DWT*round_trips_per_year


def get_transportation_cost(annual_cost, annual_cargo):
    return annual_cost/annual_cargo


def get_KB(T):  # Vertical Centre of Buoyancy
    return 0.53*T


def get_BMt(Cb, B, T):  # Metacentric Radius
    return (0.085*Cb - 0.002)*(B ** 2)/(T+Cb)


def get_KG(D):  # Vertical Centre of Gravity
    return 1 + 0.52*D


def get_GMt(KB, BMt, KG):  # Metacentric Height
    return KB + BMt - KG

## 3. Optimization

Now we proceed to apply the minimize function. A more detailed guide to its use can be found at https://realpython.com/python-scipy-cluster-optimize/. But, for the application in question, we can summarize such a function as a function that takes five (05) arguments:

1. **x0**: a list/vector defining the set of parameters (which will be changed in the optimization process) and their respective initial values (also know as the "initial guess" vector);
2. **constraints**: a list of dictionaries defining constraints for possible parameter values obtained in the optimization;
3. **bounds**: a list of limits (maximum and minimum values) for possible parameter values.
4. **fun**: a function that returns the value to be minimized;
5. **method**: the optimization method/algorithm used. In this case, we use the Sequential Least Squares Programming (SLSQP) algorithm. See more here: https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html#optimize-minimize-slsqp;

In mathematical terms, the minimize function is solving the following problem:

<br />
<img src="./images/minimize_function.png" width="500" /> 
<br />

Where $F(x)$ corresponds to the **fun** argument; $X$ corresponds to the param vector, whose order of elements follows from the definition of the initial guess, **x0**; $C_j(X) = 0$ is a given equality constraint and $C_j(X) >= 0$ an inequality constraint, which are defined in the **constraints** argument; and $XL <= x <= XU$ is the set of boundaries (L being the lower bound and U the upper one), which are defined in the **bounds** argument.

From the order of arguments above, we build each of them step-by-step for the analyzed case, starting with **x0**.

### 3.3 Initial Guess | x0  <a name="3.3"></a>

The order in which we place each parameter is arbitrary for program implementation. For organizational purposes, however, we put more fundamental ship parameters at the beginning.

In [18]:
x0 = [L, B, D, T, Cb, Vk, DWT]

It is important to note that, as we will see below, the order of parameters in the initial guess vector **x0** will be relevant in the understanding of other parameters.

### 3.2 Constraints | constraints  <a name="3.2"></a>

We define a sequence of python dictionaries, each representing a constraint of type $C_j(X) >= 0$ (in this case, we only deal with inequality constraints) and with the keys "type" (to define if it is an equality or inequality constraint) and "func" (to define the constraint itself).

In [19]:
def build_constraints():
    '''A function that builds constraints relating two or more parameters
    defined for the optimization problem.


    Returns
    -------
    out : sequence, dict
        Sequence of dictionaries, each representing a specif constraint
        and containing the keys "type" and "fun".

        "type" : {"eq", "ineq"}
            Determines if it is an equality or inequality constraint

        "fun" : callable
            The function defining the constraint. It receives a vector x with
            all parameters (in the order as defined by the initial guess, x0).
            Equality constraint means that the constraint function result is to
            be zero (fun(x) == 0) whereas inequality means that it is to be
            non-negative (f(x) >= 0).

    See Also
    --------
    https://stackoverflow.com/questions/42303470/scipy-optimize-inequality-constraint-which-side-of-the-inequality-is-considere/42304099

    Notes
    -----
    Constraints typically don't need a builder function like this and are
    more succinctly defined with "func" being a lambda function. The decision to
    create the build_constraints function was to try to make it easier to understand
    how constraints are defined in Scipy and to clarify where each constraint of the
    problem was defined.
    '''

    def L_and_B(x):
        '''L/B >= 6'''
        L, B = x[0], x[1]
        return L/B - 6

    def L_and_D(x):
        '''L/D  <= 15'''
        L, D = x[0], x[2]
        return 15 - L/D

    def L_and_T(x):
        '''L/T <= 19'''
        L, T = x[0], x[3]
        return 19 - L/T

    def T_and_DWT(x):
        '''T <= 0.45DWT^0,31'''
        T, DWT = x[3], x[6]
        return 0.45*DWT*(DWT**0.32) - T

    def T_and_D(x):
        '''T <= 0.7D + 0.7'''
        D, T = x[2], x[3]
        return 0.7*D + 0.7 - T

    def stability_constraint(x):
        '''GMt >= 0.07B'''
        B, D, T, Cb = x[1], x[2], x[3], x[4]
        KB = get_KB(T)
        BMt = get_BMt(Cb, B, T)
        KG = get_KG(D)
        GMt = get_GMt(KB, BMt, KG)
        return GMt - 0.07*B

    def froude_number_constraint(x):
        '''froude_number <= 0.32'''
        L, Vk = x[0], x[5]
        froude_number = get_froude_number(Vk, L)
        return 0.32 - froude_number

    return (
        {'type': 'ineq', 'fun': L_and_B},
        {'type': 'ineq', 'fun': L_and_D},
        {'type': 'ineq', 'fun': L_and_T},
        {'type': 'ineq', 'fun': T_and_DWT},
        {'type': 'ineq', 'fun': T_and_D},
        {'type': 'ineq', 'fun': stability_constraint},
        {'type': 'ineq', 'fun': froude_number_constraint},
    )

### 3.3 Boundaries | bounds  <a name="3.3"></a>

We define a sequence of python tuples, each representing a boundary of type $L <= x <= U$. In this case, the order of the boundaries given in the sequence must match the order of the parameters given in the initial guess vector **x0**.

In [20]:
def build_bounds():
    '''A function that defines bounds for each parameter of the optimization
   problem.


    Returns
    -------
    out : sequence, tup
        Sequence of tuples, each representing the minimum and maximum bounds
        for a specific parameter.

    Notes
    -----
    The order of the bounds of the returned sequence must match the order of
    the initial guess vector (x0) defined. The value "None" represents the
    absence of a minimum or maximum bound.
    '''

    L_bound = (0, 274.32)
    B_bound = (0, None)
    D_bound = (0, None)
    T_bound = (0, None)
    Cb_bound = (0.63, 0.75)
    Vk_bound = (14, 18)
    DWT_bound = (25000, 500000)
    return (L_bound, B_bound, D_bound, T_bound,
            Cb_bound, Vk_bound, DWT_bound)

### 3.4 Objective Function | func  <a name="3.4"></a>

A function that, from a set of parameters that can be changed (respecting constraints and limits), returns the calculated value of the transport cost, which we seek to minimize. 

In [21]:
def transportation_cost_function(x):
    '''A function that calculates the transportation cost from a set
    of parameters.

    Parameters
    -------
    x : list
        A list of parameters whose order matches what was defined in 
        the initial guess vector (x0)

    Returns
    -------
    out : float
        The value of the transportation cost calculated from given parameters
    '''
    L, B, D, T, Cb, Vk, DWT = x

    # Power
    displacement = get_displacement(L, B, T, Cb)
    V, froude_number = get_V(Vk), get_froude_number(Vk, L)
    power = get_power(Cb, displacement, V, froude_number)

    # Weights
    steel_weight = get_steel_weight(L, B, D, Cb)
    outfit_weight = get_outfit_weight(L, B, D, Cb)

    # We commented the lines containing the variables below because, although
    # they were mentioned in problem assignment, they are not used in our program.
    # machinery_weight = get_machinery_weight(power)
    # ligthship_weight = get_lightship_weight(
    #     steel_weight, outfit_weight, machinery_weight)

    # General (TODO: give a better name for this category of calculations)
    sea_days = get_sea_days(round_trip_miles, Vk)
    daily_consumption = get_daily_consumption(power)
    fuel_carried = get_fuel_carried(daily_consumption, sea_days)
    miscellaneous_DWT = get_miscellaneous_DWT(DWT)
    cargo_DWT = get_cargo_DWT(DWT, fuel_carried, miscellaneous_DWT)

    port_days = get_port_days(cargo_DWT, handling_rate)
    round_trips_per_year = get_round_trips_per_year(sea_days, port_days)

    # Costs
    port_cost = get_port_cost(DWT)
    fuel_cost = get_fuel_cost(daily_consumption, sea_days, fuel_price)
    voyage_costs = get_voyage_costs(fuel_cost, port_cost, round_trips_per_year)
    running_cost = get_running_cost(DWT)
    ship_cost = get_ship_cost(steel_weight, outfit_weight, power)
    capital_costs = get_capital_costs(ship_cost)
    annual_cost = get_annual_cost(capital_costs, running_cost, voyage_costs)
    annual_cargo = get_annual_cargo(cargo_DWT, round_trips_per_year)

    return get_transportation_cost(annual_cost, annual_cargo)


### 3.5 Results  <a name="3.5"></a>

Finally, we use the values for each parameter developed as arguments of the minimize function:

In [24]:
from scipy.optimize import minimize

result = minimize(
    x0=x0, # Initial guess vector
    fun=transportation_cost_function, # Objective function to be optimized
    method='SLSQP', # We are using the Sequential Least Squares Programming algorithm
    constraints=build_constraints(), # Sequence of dictionaries defining the constraints
    bounds=build_bounds() # Sequence of tuples defining lower and upper bounds for each parameter
)

We then get the optimized transportation cost and the corresponding ship parameters as the result of the optimization:

In [27]:
# Get the Optimized transportation cost and the changed
# ship parameters as result of the optimization
optimized_transportation_cost = result.fun
L, B, D, T, Cb, Vk, DWT = result.x

# And finnally print the obtained values
print(f"Optimized transportation cost: {optimized_transportation_cost} \n")
print(f"Length (L) after optimization: {L}")
print(f"Breadth (B) after optimization: {B}")
print(f"Depth (D) after optimization: {D}")
print(f"Draught (T) after optimization: {T}")
print(f"Block Coefficient (Cb) after optimization: {Cb}")
print(f"Speed (Vk) after optimization: {Vk}")
print(f"Deadweigth (DWT) after optimization: {DWT}")

Optimized transportation cost: 3.2402818838874525 

Length (L) after optimization: 98.83192803980378
Breadth (B) after optimization: 16.47198800663392
Depth (D) after optimization: 6.588795202653554
Draught (T) after optimization: 5.3121566418576736
Block Coefficient (Cb) after optimization: 0.7499999999999816
Speed (Vk) after optimization: 15.756864143830995
Deadweigth (DWT) after optimization: 100000.00350009301
